# Publication references by researcher

This notebook uses the [DataCite GraphQL API](https://api.datacite.org/graphql) to fetch all DataCite references of the DataCite publications authored by a particular researcher, using his/her ORCID ID.

In [ ]:
import requests
from IPython.display import display, Markdown
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

_transport = RequestsHTTPTransport(
    url='https://api.datacite.org/graphql',
    use_json=True,
)


client = Client(
    transport=_transport,
    fetch_schema_from_transport=True,
)

In [ ]:
# Generate the GraphQL query: find the researcher by ORCID ID, then fetch the first 50 publications with DataCite DOIs linked to that ORCID account.

query = gql("""
{
  person(id: "https://orcid.org/0000-0003-1419-2405") {
    id
    name
    publications(first: 50) {
      totalCount
      nodes {
        id
        relatedIdentifiers {
          relatedIdentifier
        }
      }
    }
  }
}
""")

In [ ]:
data = client.execute(query)

In [ ]:
# Get the name of the researcher with ORCID ID https://orcid.org/0000-0003-1419-2405

display(Markdown((data['person']['name'])))

In [ ]:
# Get the number of publications

data['person']['publications']['totalCount']

In [ ]:
# Generate a list of formatted citations in APA format for the publications

ids = ','.join(
    list(map(lambda s: s.replace('https://doi.org/',''), 
             list(map(lambda x : x['id'], data['person']['publications']['nodes'])))))

response = requests.get(
    'https://api.datacite.org/dois',
    params={'style': 'apa', 'page[size]': '250', 'sort': 'created', 'ids': ids},
    headers={'Accept': 'text/x-bibliography'}
)

display(Markdown('## Publications'))
display(Markdown(response.content.decode('utf-8')))